# Long Term Memory

In [6]:
import os, dotenv

dotenv.load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

In [16]:
from langchain_cohere.embeddings import  CohereEmbeddings

model = CohereEmbeddings(model="embed-multilingual-v3.0")

In [19]:
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# Set up store and memory saver
store = InMemoryStore(
    index={
        "dims": 1024,
        "embed": "cohere:embed-multilingual-v3.0",
    }
) #

In [21]:
# Create agent with memory tools
agent = create_react_agent(
    "google_genai:gemini-2.0-flash",
    tools=[
        # Configure memory tools with runtime namespace 
        create_manage_memory_tool(namespace=("memories", "216")),
        create_search_memory_tool(namespace=("memories", "216")),
    ],
    store=store,
)

In [33]:
agent.invoke({"messages": [{"role": "user", "content": "I am Aasher, remember it. ALso I like Pizza"}]})

{'messages': [HumanMessage(content='I am Aasher, remember it. ALso I like Pizza', additional_kwargs={}, response_metadata={}, id='875d5b33-14e5-48cc-b28c-e0b3cbcca7c9'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'manage_memory', 'arguments': '{"content": "The user is Aasher and he likes Pizza.", "action": "create"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b5196f8a-9d24-44b9-8788-9bec425936dd-0', tool_calls=[{'name': 'manage_memory', 'args': {'content': 'The user is Aasher and he likes Pizza.', 'action': 'create'}, 'id': 'a7ce200f-668f-47bc-a8de-041cc86999e9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 145, 'output_tokens': 16, 'total_tokens': 161, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='created memory 026a4eaa-7abf-4d2d-966d-48c23ba83f18', name='manage_memory', id='cf33f446-bac7-4d54-88dd-e218a27337b6', tool_call_id='a7ce2

In [40]:
agent.invoke({"messages": [{"role": "user", "content": "What do you know about me"}]})

{'messages': [HumanMessage(content='What do you know anout me', additional_kwargs={}, response_metadata={}, id='29e3f526-d998-4398-97ee-7b23505bcbec'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_memory', 'arguments': '{"query": "user"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-737df022-17dc-44be-9826-5d33a417aec7-0', tool_calls=[{'name': 'search_memory', 'args': {'query': 'user'}, 'id': 'cdf01453-53cb-40cd-a414-f637d8534b78', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 5, 'total_tokens': 144, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"namespace": ["memories", "216"], "key": "db9e7b9f-13b5-494c-bcef-2a21d7455c31", "value": {"content": "The user is Aasher."}, "created_at": "2025-02-15T10:03:07.796426+00:00", "updated_at": "2025-02-15T10:03:07.796426+00:00", "score": 0.4937777624591205}, {"namespace

In [37]:
# Search for semantically similar documents
store.search(
    ("memories", "216"),
    query="What are user preferences",
    limit=1)

[Item(namespace=['memories', '216'], key='026a4eaa-7abf-4d2d-966d-48c23ba83f18', value={'content': 'The user is Aasher and he likes Pizza.'}, created_at='2025-02-15T10:07:41.303316+00:00', updated_at='2025-02-15T10:07:41.303316+00:00', score=0.4310282980827379)]